First exploration of a new strategy: Inside Bar Momentum.  Simply:  if high-low of bar 2 within high-low of bar 1: signal...

In [24]:
import pandas as pd
import plotly.graph_objects as go
import utils

We know we are going to look for data on USD_JPY_H4, but we haven't yet tested collect_his_data with this pairing, so we need to run that first.  Might as well refresh while we're in the area...

But, this takes a long time to run - so comment out after doing it once.

In [25]:
# import os
# %run ./collect_his_data.py


In [26]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))


In [27]:
df_raw.describe()

,Unnamed: 0,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
count,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000,4675.000000
mean,667.013904,17061.963422,116.006735,116.177761,115.836413,116.015489,116.019676,116.186209,115.847156,116.025663,116.032627,116.196451,115.855979,116.035833
std,419.124282,13381.055089,12.734903,12.797093,12.658796,12.737169,12.735869,12.797949,12.659539,12.737199,12.736837,12.798409,12.660817,12.737247
min,0.000000,8.000000,102.064000,102.556000,101.170000,102.063000,102.072000,102.566000,101.180000,102.070000,102.079000,102.579000,101.190000,102.077000
25%,292.000000,7905.500000,107.191000,107.317000,107.072000,107.195000,107.202500,107.325000,107.086000,107.206000,107.215000,107.334000,107.094000,107.217500
50%,643.000000,13316.000000,109.985000,110.099000,109.884000,109.995000,110.002000,110.107000,109.892000,110.001000,110.010000,110.117000,109.898000,110.007000
75%,1033.000000,21582.000000,122.641500,122.949000,122.472000,122.697000,122.648000,122.957000,122.482000,122.705000,122.665500,122.966000,122.492000,122.712500
max,1424.000000,114647.000000,151.467000,151.938000,150.767000,151.465000,151.480000,151.946000,150.780000,151.478000,151.494000,151.956000,150.793000,151.492000


In [28]:
non_nums = ['ticker', 'time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)



In [29]:
df_raw.columns

Index(['Unnamed: 0', 'ticker', 'time', 'volume', 'bid_o', 'bid_h', 'bid_l',
       'bid_c', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_o', 'ask_h', 'ask_l',
       'ask_c'],
      dtype='object')

In [30]:
df = df_raw[['ticker', 'time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c']].copy()

In [31]:
def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

In [32]:
def get_signal(row):
    """  Identify encapsulation in previous candle and return direction.  Pass back zero if no encapsulation.  
    1 = Buy, -1=Sell:  Buy if encapsulated in a previously upward candle...      """
    if row.mid_h_prev > row.mid_h and row.mid_l_prev > row.mid_l:
        return row.DIRECTION_prev
    return 0

## Additional column capture/derivation.

* The range of a candle is the difference between the high and low price.
* To track whether the high and low and enclosed in previous candle, we need high and low values from previous.



In [33]:
#TODO:  Create a new dataframe/sheet to capture Data Dictionary and some metadata...

df = df_raw[['ticker', 'time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)
df.head()


,ticker,time,mid_o,mid_h,mid_l,mid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL
0,USD_JPY,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.747,0.100,108.761,108.607,0.154,1,1,0
1,USD_JPY,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.845,0.146,108.758,108.658,0.100,1,1,0
2,USD_JPY,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.700,0.208,108.842,108.696,0.146,-1,1,0
3,USD_JPY,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.535,0.536,108.866,108.658,0.208,-1,-1,-1
4,USD_JPY,2020-01-02T18:00:00.000000000Z,108.526,108.572,108.494,108.561,108.611,0.078,108.748,108.212,0.536,1,-1,0


Capture how many rows in our dataframe, and count how many buy/sell signals we're getting.

In [34]:
print(f"total number of rows:  {df.shape[0]}")
df.groupby(by="SIGNAL").count()

total number of rows:  4674


,ticker,time,mid_o,mid_h,mid_l,mid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev
SIGNAL,,,,,,,,,,,,,
-1,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112
0,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161
1,401,401,401,401,401,401,401,401,401,401,401,401,401


In [35]:
df_plot = df.iloc[200:300]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1
                   ]

In [36]:

fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
    ))

fig.add_trace(go.Scatter(
    x=df_buys.time,
    y=df_buys.mid_c,
    mode='markers',
    marker=dict(color="#0ec4ce", size=12),
    
))
fig.add_trace(go.Scatter(
    x=df_sells.time,
    y=df_sells.mid_c,
    mode='markers',
    marker=dict(color="#043ef9", size=12)
))


fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                margin=dict(l=10, b=10, t=10, r=10), 
                font=dict(size=10, color="#e1e1e1"))
fig.update_xaxes(gridcolor="#1f292f",
                showgrid=True,
                fixedrange=True,
                rangeslider=dict(visible=False),
                rangebreaks=[
                    dict(bounds=["sat", "mon"])
                    ]
                )
fig.update_yaxes(gridcolor="#1f292f",
                showgrid=True)


In [38]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    """  Identify encapsulation in previous candle and return direction.  Pass back zero if no encapsulation.  
    1 = Buy, -1=Sell:  Buy if encapsulated in a previously upward candle...      """
    if row.mid_h_prev > row.mid_h and row.mid_l_prev > row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.mid_h_prev
    elif row.SIGNAL == -1:
        return row.mid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.mid_h_prev - (row.RANGE_prev * SLOSS)
    if row.SIGNAL == -1:
        return row.mid_h_prev + (row.RANGE_prev * SLOSS)
    else:
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.mid_h_prev + (row.RANGE_prev * TPROFIT)
    if row.SIGNAL == -1:
        return row.mid_h_prev - (row.RANGE_prev * TPROFIT)
    else:
        return 0

In [42]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)
df[df.SIGNAL==1].head()


,ticker,time,mid_o,mid_h,mid_l,mid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
11,USD_JPY,2020-01-05T22:00:00.000000000Z,107.879,108.122,107.772,107.948,107.954,0.350,108.126,107.842,0.284,1,1,1,108.1544,108.0124,108.3532
17,USD_JPY,2020-01-06T22:00:00.000000000Z,108.353,108.461,108.330,108.434,108.439,0.131,108.506,108.342,0.164,1,1,1,108.5224,108.4404,108.6372
19,USD_JPY,2020-01-07T06:00:00.000000000Z,108.438,108.500,108.262,108.468,108.475,0.238,108.506,108.423,0.083,1,1,1,108.5143,108.4728,108.5724
34,USD_JPY,2020-01-09T18:00:00.000000000Z,109.510,109.538,109.392,109.524,109.532,0.146,109.580,109.429,0.151,1,1,1,109.5951,109.5196,109.7008
62,USD_JPY,2020-01-16T10:00:00.000000000Z,110.002,110.054,109.929,110.013,110.019,0.125,110.066,109.932,0.134,1,1,1,110.0794,110.0124,110.1732
